<a href="https://colab.research.google.com/github/sharmin6630/Python-for-Everybody/blob/master/Using%20Databases%20with%20Python/Data_Models_and_Relational_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Models and Relational SQL**

# **Assignment**

**Musical Track Database**

```
To get credit for this assignment, perform the instructions below and upload 
your SQLite3 database here:
CREATE TABLE Artist (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Genre (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Album (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    artist_id  INTEGER,
    title   TEXT UNIQUE
);

CREATE TABLE Track (
    id  INTEGER NOT NULL PRIMARY KEY 
        AUTOINCREMENT UNIQUE,
    title TEXT  UNIQUE,
    album_id  INTEGER,
    genre_id  INTEGER,
    len INTEGER, rating INTEGER, count INTEGER
);
```


```
If you run the program multiple times in testing or with different files, make 
sure to empty out the data before each run.

You can use this code as a starting point for your application: http://www.py4e.com/code3/tracks.zip. 
The ZIP file contains the Library.xml file to be used for 
this assignment. You can export your own tracks from iTunes and create a 
database, but for the database that you turn in for this assignment, only use 
the Library.xml data that is provided.

To grade this assignment, the program will run a query like this on your 
uploaded database and look for the data it expects to see:

SELECT Track.title, Artist.name, Album.title, Genre.name 
    FROM Track JOIN Genre JOIN Album JOIN Artist 
    ON Track.genre_id = Genre.ID and Track.album_id = Album.id 
        AND Album.artist_id = Artist.id
    ORDER BY Artist.name LIMIT 3
```




In [14]:
import xml.etree.ElementTree as ET
import sqlite3

conn = sqlite3.connect('trackdb.sqlite')
cur = conn.cursor()

# Make some fresh tables using executescript()
cur.executescript('''
DROP TABLE IF EXISTS Artist;
DROP TABLE IF EXISTS Album;
DROP TABLE IF EXISTS Track;
DROP TABLE IF EXISTS Genre;

CREATE TABLE Artist (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Genre (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Album (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    artist_id  INTEGER,
    title   TEXT UNIQUE
);

CREATE TABLE Track (
    id  INTEGER NOT NULL PRIMARY KEY 
        AUTOINCREMENT UNIQUE,
    title TEXT  UNIQUE,
    album_id  INTEGER,
    genre_id  INTEGER,
    len INTEGER, rating INTEGER, count INTEGER
);
''')


fname = input('Enter file name: ')
if len(fname) < 1 :
  fname = 'Library.xml'

# <key>Track ID</key><integer>369</integer>
# <key>Name</key><string>Another One Bites The Dust</string>
# <key>Artist</key><string>Queen</string>
# <key>Composer</key><string>John Deacon</string>
# <key>Album</key><string>Greatest Hits</string>
# <key>Genre</key><string>Rock</string>

def lookup(record, key_name):
  found = False
  for child in record:
    if found : 
      return child.text #sending value of the found key
    if child.tag == 'key' and child.text == key_name :
        found = True
  return None

stuff = ET.parse(fname)
all = stuff.findall('dict/dict/dict')
print('Dict count:', len(all))
for record in all:
  if ( lookup(record, 'Track ID') is None ) : 
    continue
  #now looking for other needed key values
  name = lookup(record, 'Name')
  artist = lookup(record, 'Artist')
  album = lookup(record, 'Album')
  genre = lookup(record, 'Genre')
  count = lookup(record, 'Play Count')
  rating = lookup(record, 'Rating')
  length = lookup(record, 'Total Time')

  if name is None or artist is None or album is None or genre is None: 
    continue

  #print(name, artist, album, genre, count, rating, length)

  cur.execute('''INSERT OR IGNORE INTO Artist (name) 
    VALUES ( ? )''', ( artist, ) )
  cur.execute('SELECT id FROM Artist WHERE name = ? ', (artist, ))
  artist_id = cur.fetchone()[0]

  cur.execute('''INSERT OR IGNORE INTO Album (artist_id, title) 
    VALUES ( ?, ? )''', ( artist_id, album ) )
  cur.execute('SELECT id FROM Album WHERE title = ? ', (album, ))
  album_id = cur.fetchone()[0]

  cur.execute('''INSERT OR IGNORE INTO Genre (name) 
    VALUES ( ? )''', (genre, ) )
  cur.execute('SELECT id FROM Genre WHERE name = ? ', (genre, ))
  genre_id = cur.fetchone()[0]

  cur.execute('''INSERT OR REPLACE INTO Track
    (title, album_id, genre_id, len, rating, count) 
    VALUES ( ?, ?, ?, ?, ?, ? )''', 
    ( name, album_id, genre_id, length, rating, count ) )

conn.commit()

sqlstr = '''SELECT Track.title, Artist.name, Album.title, Genre.name 
    FROM Track JOIN Genre JOIN Album JOIN Artist 
    ON Track.genre_id = Genre.id and Track.album_id = Album.id AND Album.artist_id = Artist.id
    ORDER BY Artist.name LIMIT 10'''

for row in cur.execute(sqlstr):
    print(str(row[0]), row[1], row[2], row[3])

cur.close()

Enter file name: /content/sample_data/Library.xml
Dict count: 404
For Those About To Rock (We Salute You) AC/DC Who Made Who Rock
Hells Bells AC/DC Who Made Who Rock
Shake Your Foundations AC/DC Who Made Who Rock
You Shook Me All Night Long AC/DC Who Made Who Rock
Who Made Who AC/DC Who Made Who Rock
D.T. AC/DC Who Made Who Rock
Sink the Pink AC/DC Who Made Who Rock
Ride On AC/DC Who Made Who Rock
Chase the Ace AC/DC Who Made Who Rock
Track 01 Billy Price Danger Zone Blues/R&B
